In [1]:
import tensorflow as tf

print(f'tensorflow version {tf.__version__}')

from imblearn.over_sampling import SMOTE
from collections import Counter
from numpy import * 

import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

tensorflow version 2.9.1


# Loading the Data & Smote

In [2]:
def dataLoadSm():
    test = pd.read_csv('Real_Tomorrow/test_real_tom_target.csv')
    train = pd.read_csv('Real_Tomorrow/training_real_tom_target.csv')
    val = pd.read_csv('Real_Tomorrow/validation_real_tom_target.csv')
    print(train.head())
    
    #Split data
    x_train = train[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******training features******")
    print (x_train)
    y_train = train[['Real_tom_lsm']]
    print ("******training target******")
    print (y_train)
    
    x_test = test[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******test features******")
    print (x_test)
    y_test = test[['Real_tom_lsm']]
    print ("******test target******")
    print (y_test)
    
    x_val = val[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******validation features******")
    print (x_val)
    y_val = val[['Real_tom_lsm']]
    print ("******validation target******")
    print (y_val)
    
    print('Before Smote')

    dff = y_train
    M = y_train.to_numpy()
    # summarize dataset
    classes = unique(M)
    print(classes)
    total = len(M)
    for c in classes:
        n_examples = len(M[M==c])
        percent = n_examples / total * 100
        print('> Class=%d : %d/%d (%.1f%%)' % (c, n_examples, total, percent))
    
    
    smt = SMOTE()
    
    X_train_sm, Y_train_sm = smt.fit_resample(x_train, y_train)
    
    print('After SMOTE')
    dff = Y_train_sm
    M = Y_train_sm.to_numpy()
    # summarize dataset
    classes = unique(M)
    print(classes)
    total = len(M)
    for c in classes:
        n_examples = len(M[M==c])
        percent = n_examples / total * 100
        print('> Class=%d : %d/%d (%.1f%%)' % (c, n_examples, total, percent))
        
    return X_train_sm, Y_train_sm, x_val, y_val, x_test, y_test

def dataLoad():
    test = pd.read_csv('Real_Tomorrow/test_real_tom_target.csv')
    train = pd.read_csv('Real_Tomorrow/training_real_tom_target.csv')
    val = pd.read_csv('Real_Tomorrow/validation_real_tom_target.csv')
    print(train.head())
    
    #Split data
    x_train = train[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******training features******")
    #print (x_train)
    y_train = train[['Real_tom_lsm']]
    print ("******training target******")
    #print (y_train)
    
    x_test = test[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******test features******")
    #print (x_test)
    y_test = test[['Real_tom_lsm']]
    print ("******test target******")
    #print (y_test)
    
    x_val = val[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******validation features******")
    #print (x_val)
    y_val = val[['Real_tom_lsm']]
    print ("******validation target******")
    #print (y_val)
    
        
    return x_train, y_train, x_val, y_val, x_test, y_test

In [5]:
x_train_sm, y_train_sm, x_val, y_val, x_test, y_test = dataLoadSm()


   Unnamed: 0        time  latitude  longitude        vo          r     u_200  \
0           0  1980-01-01       0.0       20.0  0.000007  80.761185  1.909660   
1           1  1980-01-01       0.0       22.5  0.000004  80.703650  1.165733   
2           2  1980-01-01       0.0       25.0  0.000007  78.231514 -1.311676   
3           3  1980-01-01       0.0       27.5  0.000010  79.631010 -3.777573   
4           4  1980-01-01       0.0       30.0  0.000010  71.573875 -5.734505   

      u_850     v_200     v_850        ttr  sst  lsm  Real_tom_lsm  
0 -3.323872  1.687164 -1.823624 -247.54074  0.0  0.0           0.0  
1 -2.844494  1.060593 -1.991425 -240.00592  0.0  0.0           0.0  
2 -2.125244  3.280617 -1.931789 -223.76889  0.0  0.0           0.0  
3 -1.122395  5.743889 -1.243538 -235.55556  0.0  0.0           0.0  
4 -1.362953  6.514030 -0.954163 -254.03260  0.0  0.0           0.0  
******training features******
               vo          r      u_200     u_850      v_200     v_85

# Standardization

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

#scaled_x_training = scaler.fit_transform(x_training)
#df = pd.DataFrame(StandardScaler().fit_transform(x_training))

X_train_stand = x_train_sm.copy() #smote
X_valid_stand = x_val.copy() #smote
X_test_stand = x_test.copy() #smote

#not smote
X_train_scaled = x_train.copy() 
X_valid_scaled = x_val.copy()
X_test_scaled = x_test.copy()

num_cols = [ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']

# apply standardization on numerical features
for i in num_cols:
    
    # fit on training data column
    scale = StandardScaler().fit(X_train_stand[[i]])
    
    # transform the training data column
    X_train_stand[i] = scale.transform(X_train_stand[[i]])
    
    # transform the training data column
    X_valid_stand[i] = scale.transform(X_valid_stand[[i]])   

    # transform the testing data column
    X_test_stand[i] = scale.transform(X_test_stand[[i]])
    
for i in num_cols:
    
    # fit on training data column
    scale = StandardScaler().fit(X_train_scaled[[i]])
    
    # transform the training data column
    X_train_scaled[i] = scale.transform(X_train_scaled[[i]])
    
    # transform the training data column
    X_valid_scaled[i] = scale.transform(X_valid_scaled[[i]])   

    # transform the testing data column
    X_test_scaled[i] = scale.transform(X_test_scaled [[i]])

# LogisticRegression

In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

def plot_roc(pred,y):
    fpr, tpr, _ = roc_curve(y, pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()

def logistic_regression_with_val(x,y,x_val,y_val):
    for C in [100, 10, 1.0, 0.1, 0.01]:
    #for C in [0.001, 0.0001]:
    #for C in [0.00001]:
        for penalty in ['l1','l2','elasticnet','none']:
            for solver in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
                try:
                    print(f'##########################Penalty:{penalty}, solver:{solver}, C{C}##########################')
                    clf = LogisticRegression(random_state=0, penalty=penalty, solver=solver, C=C).fit(x, y)
                    print(clf.coef_)
                    print(clf.intercept_)
                    
                    #preds = clf.predict(x_val)
                    #plot_roc(preds,y_val)
    
                    conf = confusion_matrix(y_val, np.round(preds))
                    np.set_printoptions(precision=2)
                    print(conf)  
                except: continue
                    
def logistic_regression_with_val_class_weight(x,y,x_val,y_val):
    for C in [100, 10, 1.0, 0.1, 0.01]:
    #for C in [0.001, 0.0001]:
    #for C in [0.00001]:
        for penalty in ['l1','l2','elasticnet','none']:
            for solver in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
                try:
                    print(f'##########################Penalty:{penalty}, solver:{solver}, C{C}##########################')
                    clf = LogisticRegression(random_state=0, penalty=penalty, class_weight='balanced', solver=solver, C=C).fit(x, y)
                    print(clf.coef_)
                    print(clf.intercept_)
                    
                    #preds = clf.predict(x_val)
                    #plot_roc(preds,y_val)
    
                    conf = confusion_matrix(y_val, np.round(preds))
                    np.set_printoptions(precision=2)
                    print(conf)  
                except: continue

# Modeling_Scaled_SMOTE


In [9]:
logistic_regression_with_val(X_train_stand,y_train_sm,X_valid_stand,y_val)

##########################Penalty:l1, solver:newton-cg, C100##########################
##########################Penalty:l1, solver:lbfgs, C100##########################
##########################Penalty:l1, solver:liblinear, C100##########################


/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[-1.47699488  0.36084466 -0.1655262  -1.01968242 -0.05790316 -0.02690608
   1.40204142  0.62883605]]
[0.5189744]
##########################Penalty:l1, solver:sag, C100##########################
##########################Penalty:l1, solver:saga, C100##########################


/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[-1.47722563  0.36073075 -0.16555158 -1.0198096  -0.05784455 -0.02698304
   1.40226524  0.62882665]]
[0.51914743]
##########################Penalty:l2, solver:newton-cg, C100##########################


...

[[-1.4772021   0.36070985 -0.16556998 -1.01976577 -0.05786534 -0.02694852
   1.40224427  0.6288075 ]]
[0.51910488]
##########################Penalty:none, solver:liblinear, C0.01##########################
##########################Penalty:none, solver:sag, C0.01##########################


/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[-1.47729718  0.36074274 -0.16553821 -1.01979859 -0.05781527 -0.0270265
   1.40226751  0.62875695]]
[0.51909467]
##########################Penalty:none, solver:saga, C0.01##########################


/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[-1.4772257   0.36073075 -0.16555159 -1.01980964 -0.05784456 -0.02698307
   1.40226526  0.62882666]]
[0.51914745]


# loading data

In [3]:
x_train, y_train, x_val, y_val, x_test, y_test = dataLoad()


   Unnamed: 0        time  latitude  longitude        vo          r     u_200  \
0           0  1980-01-01       0.0       20.0  0.000007  80.761185  1.909660   
1           1  1980-01-01       0.0       22.5  0.000004  80.703650  1.165733   
2           2  1980-01-01       0.0       25.0  0.000007  78.231514 -1.311676   
3           3  1980-01-01       0.0       27.5  0.000010  79.631010 -3.777573   
4           4  1980-01-01       0.0       30.0  0.000010  71.573875 -5.734505   

      u_850     v_200     v_850        ttr  sst  lsm  Real_tom_lsm  
0 -3.323872  1.687164 -1.823624 -247.54074  0.0  0.0           0.0  
1 -2.844494  1.060593 -1.991425 -240.00592  0.0  0.0           0.0  
2 -2.125244  3.280617 -1.931789 -223.76889  0.0  0.0           0.0  
3 -1.122395  5.743889 -1.243538 -235.55556  0.0  0.0           0.0  
4 -1.362953  6.514030 -0.954163 -254.03260  0.0  0.0           0.0  
******training features******
******training target******
******test features******
******test targ

# Modeling_Scaled_class_weight


In [19]:
logistic_regression_with_val_class_weight(X_train_scaled,y_train,X_valid_scaled,y_val)

##########################Penalty:l1, solver:newton-cg, C100##########################
##########################Penalty:l1, solver:lbfgs, C100##########################
##########################Penalty:l1, solver:liblinear, C100##########################


/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[-0.41705726  0.28400323  0.14547696 -0.68773563 -0.03177079  0.07213172
   0.88752071  0.67371025]]
[-7.5923385]
##########################Penalty:l1, solver:sag, C100##########################
##########################Penalty:l1, solver:saga, C100##########################


/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[-0.4170667   0.28398066  0.14547838 -0.68774149 -0.03177496  0.07213486
   0.88754934  0.67370835]]
[-7.59234049]
##########################Penalty:l2, solver:newton-cg, C100##########################


...

[[-0.41705441  0.28396706  0.14552268 -0.68776078 -0.03170306  0.07217815
   0.88755953  0.67370496]]
[-7.59236383]
##########################Penalty:none, solver:liblinear, C0.01##########################
##########################Penalty:none, solver:sag, C0.01##########################


/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[-0.41706423  0.28396452  0.14548065 -0.68771896 -0.03177032  0.07215606
   0.88755692  0.67367951]]
[-7.59229183]
##########################Penalty:none, solver:saga, C0.01##########################


/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[-0.41706668  0.28398474  0.14548269 -0.68774176 -0.03177587  0.07213654
   0.88754942  0.67371242]]
[-7.59234469]


In [9]:
logistic_regression_with_val_class_weight(X_train_scaled,y_train,X_valid_scaled,y_val)

##########################Penalty:l1, solver:newton-cg, C100##########################
##########################Penalty:l1, solver:lbfgs, C100##########################
##########################Penalty:l1, solver:liblinear, C100##########################


/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[-0.43618154  0.38734269 -0.17907799 -0.66831736 -0.07788279 -0.01090204
   0.87892089  0.77448142]]
[-1.63011045]
##########################Penalty:l1, solver:sag, C100##########################
##########################Penalty:l1, solver:saga, C100##########################


/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[-1.27932123  0.81246006 -0.32050952 -1.20584754  0.23619112  0.01403104
   1.34218486  1.05742121]]
[-1.80117173]
##########################Penalty:l2, solver:newton-cg, C100##########################
[[-0.43618169  0.38734221 -0.17907801 -0.66831755 -0.07788276 -0.01090211
   0.87892147  0.77448148]]
[-1.63011069]
##########################Penalty:l2, solver:lbfgs, C100##########################


...

/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[-0.43617556  0.38736898 -0.1790581  -0.66831266 -0.07787275 -0.01089622
   0.87891137  0.77449133]]
[-1.63010395]
##########################Penalty:none, solver:liblinear, C0.01##########################
##########################Penalty:none, solver:sag, C0.01##########################


/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/utils/validation.py

[[-4.45039987 -3.76754577  0.31472299 -3.4650241   0.27616932  1.12625655
   8.02794859  1.87860467]]
[-2.58163695]
##########################Penalty:none, solver:saga, C0.01##########################
[[-1.37620295  0.51153164  0.00200825 -1.84014012 -0.03388191 -0.16704782
   1.71841366  1.17882738]]
[-2.34738539]


/Users/3rfanian/opt/anaconda3/envs/SHAP/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
